In [1]:
import Metashape
import numpy as np

In [2]:
demo_ms = '/home/crest/z/hwang_Pro/data/2023_hokkaido_potato/projects.psx/test_1R_Group0.psx'

In [3]:
doc = Metashape.Document()
doc.open(demo_ms)

LoadProject: path = /home/crest/z/hwang_Pro/data/2023_hokkaido_potato/projects.psx/test_1R_Group0.psx
loaded project in 0.002937 sec


Document.open(): The document is opened in read-only mode because it is already in use.


In [4]:
chunk = doc.chunk

In [5]:
T = chunk.transform.matrix

In [8]:
marker_xyz_dict = {}
total_points = []
for m in chunk.markers:
    if m.label in ['51', '52', '54']:
        # marker_xyz_dict[m.label] = m.position
        crs_position = chunk.crs.project(T.mulp(m.position))
        marker_xyz_dict[m.label] = crs_position
        total_points.append(crs_position)

marker_xyz_dict

{'51': Vector([0.358949897390428, -0.15629292294798577, -1.1117442613335027]),
 '52': Vector([0.31636411065873593, -0.15212397770115713, -1.0405643339520458]),
 '54': Vector([0.4271120406462694, -0.158958995683609, -1.0675086945740448])}

In [23]:
m.reference.location

In [9]:
marker_xyz = np.asarray(total_points).astype(np.float64)
marker_xyz

array([[ 0.3589499 , -0.15629292, -1.11174426],
       [ 0.31636411, -0.15212398, -1.04056433],
       [ 0.42711204, -0.158959  , -1.06750869]])

Not use '53' because its position changed since 2R1-10

In [10]:
def points2plane(points):
    # ref: https://math.stackexchange.com/questions/99299/best-fitting-plane-given-a-set-of-points
    A = np.matrix(np.insert(points[:,0:2], obj=2, values=1, axis=1))
    b = np.matrix(points[:,2]).T
  
    fit = (A.T * A).I * A.T * b
    #errors = b - A * fit
    #residual = np.linalg.norm(errors)
  
    return float(fit[0]), float(fit[1]), -1.0, float(fit[2])  # Ax+By+Cz+D=0

def point_on_plane(points, plane_param):
    # ref: https://blog.csdn.net/soaryy/article/details/82884691
    # Plane_params = (A, B, C ,D) -> Ax + By + Cz + D =0
    A, B, C, D = plane_param
    demomin = A**2 + B**2 + C**2
    x0, y0, z0 = points[:,0], points[:,1], points[:,2]
  
    xp = ((B**2+C**2)*x0 - A*(B*y0+C*z0+D)) / demomin
    yp = ((A**2+C**2)*y0 - B*(A*x0+C*z0+D)) / demomin
    zp = ((A**2+B**2)*z0 - C*(A*x0+B*y0+D)) / demomin
  
    return np.vstack([xp, yp, zp]).T


In [11]:
params = points2plane(marker_xyz)
print(f"{params[0]}x+{params[1]}y+{params[2]}z+{params[3]}=0")

2.1930032087858073x+39.475379164118294y+-1.0z+4.270799855934257=0


/tmp/ipykernel_1734477/180089087.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(fit[0]), float(fit[1]), -1.0, float(fit[2])  # Ax+By+Cz+D=0


In [12]:
# project plots boundary to point plane
plane_points = point_on_plane(marker_xyz, params)
plane_points

array([[ 0.3589499 , -0.15629292, -1.11174426],
       [ 0.31636411, -0.15212398, -1.04056433],
       [ 0.42711204, -0.158959  , -1.06750869]])

In [13]:
def vector_mod(Ax, By, Cz):
    return np.sqrt(np.sum(np.square([Ax, By, Cz])))

def find_unit_orthogonal_set(gcp3, params):
    v1 = marker_xyz_dict['51'] - marker_xyz_dict['52']
    v3 = -np.asarray(params[0:3]) # 这边要看z向量的方向是否一致，修改uv3前面的正负号
    uv1 = v1 / vector_mod(*v1)
    uv3 = v3 / vector_mod(*v3)
    uv2 = np.cross(uv1, uv3)
    
    return np.vstack([uv1, uv2, uv3]).T   # [v1, v2, v3] -> x, y, z coordinate

def trans_coord(cvtmat, points):
    return np.linalg.inv(cvtmat).dot(points.T).T


In [14]:
cvtmat = find_unit_orthogonal_set(plane_points, params)
cvtmat

array([[ 0.5127651 , -0.85673636, -0.05545043],
       [-0.05019726,  0.03455901, -0.99814123],
       [-0.8570602 , -0.51459545,  0.02528516]])

In [15]:
plot_trans = trans_coord(cvtmat, plane_points)
plot_trans

array([[1.14473421, 0.25917178, 0.10798785],
       [1.06168296, 0.25917178, 0.10798785],
       [1.14190667, 0.17791923, 0.10798785]])

In [16]:
plot_trans - plot_trans.mean(axis=0)

array([[ 2.86262678e-02,  2.70841817e-02, -2.45636844e-15],
       [-5.44249899e-02,  2.70841817e-02,  4.67681449e-15],
       [ 2.57987221e-02, -5.41683633e-02, -2.22044605e-15]])

In [17]:
np.set_printoptions(suppress=True)

calculate the center points

In [18]:
x = (plot_trans[0,0] + plot_trans[1,0]) / 2  # 51-52
y = (plot_trans[0,1] + plot_trans[-1,1]) / 2  # 51-54
z = plot_trans[-1,-1]

In [19]:
# plot_trans_ctr = plot_trans - plot_trans.mean(axis=0)
plot_trans_ctr = plot_trans - np.array([x,y,z])
plot_trans_ctr

array([[ 0.04152563,  0.04062627, -0.        ],
       [-0.04152563,  0.04062627,  0.        ],
       [ 0.03869808, -0.04062627,  0.        ]])

In [20]:
np.array([51,52,54]).reshape(3,1)

array([[51],
       [52],
       [54]])

In [21]:
plot_trans_csv = np.hstack([np.array([51,52,54]).reshape(3,1), plot_trans_ctr])
plot_trans_csv

array([[51.        ,  0.04152563,  0.04062627, -0.        ],
       [52.        , -0.04152563,  0.04062627,  0.        ],
       [54.        ,  0.03869808, -0.04062627,  0.        ]])

In [22]:
np.savetxt('gcp.csv', plot_trans_csv, delimiter= ',', fmt='%i,%f,%f,%f')

manually remove 51.0 -> 51

In [90]:
x0, y0, z0 = plot_trans_ctr[0,:]
x1, y1, z1 = plot_trans_ctr[1,:]

In [91]:
np.sqrt((x0 - x1) ** 2 + (y0 - y1)**2)

0.3229498386998291